<a href="https://colab.research.google.com/github/XiaoLiang28/Datamanagement/blob/main/PS2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set Up

In [1]:
#---------------------------SETUP----------------------------------
#get useful libraries
import time, os, sys, re #basics
import zipfile, json, datetime, string   #string for annotating points in scatter
import numpy as np #basic math
from statistics import * #stats

import matplotlib.pyplot as plt #import pylab as plt #apparently discouraged now:
 #https://stackoverflow.com/questions/11469336/what-is-the-difference-between-pylab-and-pyplot
 #https://www.tutorialspoint.com/matplotlib/matplotlib_pylab_module.htm

import pandas as pd
import pandas_datareader as pdr
from pandas_datareader import wb
from pandas.io.formats.style import Styler
#s4 = Styler(df4, uuid_len=0, cell_ids=False)

import urllib  #weird, guess need to have os and pandas imported for this to work  %TODO/LATER ditch it, its weird anyway, just use wget/curl

from google.colab import files

#import webbrowser

import seaborn as sns

from google.colab import data_table
data_table.enable_dataframe_formatter() #this enables spreadsheet view upon calling dataframe (without() )

#many tricks how to extend notebook functionality
#https://coderzcolumn.com/tutorials/python/list-of-useful-magic-commands-in-jupyter-notebook-lab
#will display all output not just last command
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#MAGICS and THEMES/STYLES: important! does affect not just shading/colors, but also fonts, spacing, etc
#(even if you only select default (v not selecting anything) [but does seem to work better if you do make explicit sleections])

###magics: https://ipython.readthedocs.io/en/stable/interactive/magics.html
#most essential setup for vis: it does affect vis! careful!! stick with inline, maybe notebook; others mostly for non-notebook, eg spyder environ
#https://jakevdp.github.io/PythonDataScienceHandbook/04.00-introduction-to-matplotlib.html recomends *inline*!
#show current one:
#%matplotlib
#%matplotlib --list
#interactive plots:
#%matplotlib notebook
#static images of your plot:
%matplotlib inline
#may play with this one and other magics (btw default is probably agg)
#%matplotlib nbagg

###themes/styles: https://matplotlib.org/stable/gallery/style_sheets/style_sheets_reference.html
#https://jakevdp.github.io/PythonDataScienceHandbook/04.11-settings-and-stylesheets.html
#https://matplotlib.org/stable/tutorials/introductory/customizing.html
#here more about art and style than under the hood functionality as with magics, explore and experiment
#many may find 'default' or seaborn ones more pleasing; my fav 'classic' is back from 90s ;)
#plt.style.available #list available styles :) may install more
#plt.style.use('default') # more delicate subtle than classic
plt.style.use('classic')  #  'seaborn-whitegrid' 'seaborn-white' 'seaborn-poster'
# btw: magics v theme/style sequence matters, eg if i specify classic style before inline magic, i wouldnt get grey bounding box im getting

# 1. Dataset 1 Import

In [2]:
df1=pd.read_csv("https://github.com/XiaoLiang28/Datamanagement/raw/main/enrollment_2021.csv")

In [3]:
df1

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41
0,2020-2021 Enrollment Data - District Counts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,County Code,County Name,District Code,District Name,Total Enrollment,White,%White,Black,%Black,Hispanic,...,Tenth Grade,Eleventh Grade,Twelfth Grade,Ungraded,%Free Lunch,%Reduced Lunch,%English Learners,%Migrant,%Military,%Homeless
2,01,Atlantic,0010,Absecon Public Schools District,887,447,50.4,133,15,208,...,0,0,0,25,40.8,5.9,8.7,0,0.8,0.2
3,01,Atlantic,0110,Atlantic City School District,6450,265,4.1,2147,33.3,2847,...,462,405,369,286,81.5,3.6,16.1,0,0,2.2
4,01,Atlantic,0120,Atlantic County Vocational School District,1713,516,30.1,296,17.3,721,...,399,355,362,160,48.6,9.6,0,0,2.5,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668,80,Charters,8050,Unity Charter School,245,123,50.2,40,16.3,28,...,0,0,0,0,9.8,1.6,1.2,0,0,0
669,80,Charters,8060,University Academy Charter High School,439,18,4.1,240,54.7,142,...,118,105,100,0,53.3,9.8,0,0,0,0
670,80,Charters,8065,University Heights Charter School,858,0,0,724,84.4,133,...,0,0,0,0,73.1,13.9,6.4,0,0,0
671,80,Charters,8140,The Village Charter School,363,4,1.1,345,95,14,...,0,0,0,0,64.5,5.5,0,0,0,0


In [4]:
df1.drop([0,672],inplace=True)

In [5]:
df1

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41
1,County Code,County Name,District Code,District Name,Total Enrollment,White,%White,Black,%Black,Hispanic,...,Tenth Grade,Eleventh Grade,Twelfth Grade,Ungraded,%Free Lunch,%Reduced Lunch,%English Learners,%Migrant,%Military,%Homeless
2,01,Atlantic,0010,Absecon Public Schools District,887,447,50.4,133,15,208,...,0,0,0,25,40.8,5.9,8.7,0,0.8,0.2
3,01,Atlantic,0110,Atlantic City School District,6450,265,4.1,2147,33.3,2847,...,462,405,369,286,81.5,3.6,16.1,0,0,2.2
4,01,Atlantic,0120,Atlantic County Vocational School District,1713,516,30.1,296,17.3,721,...,399,355,362,160,48.6,9.6,0,0,2.5,0.6
5,01,Atlantic,0125,Atlantic County Special Services School District,306,95,31,82,26.8,108,...,3,5,13,244,39.5,5.9,0.3,0,0.3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,80,Charters,8010,Union County Teams Charter School-High School/...,400,1,0.3,242,60.5,153,...,33,34,39,0,61.3,10,4,0,0,4.8
668,80,Charters,8050,Unity Charter School,245,123,50.2,40,16.3,28,...,0,0,0,0,9.8,1.6,1.2,0,0,0
669,80,Charters,8060,University Academy Charter High School,439,18,4.1,240,54.7,142,...,118,105,100,0,53.3,9.8,0,0,0,0
670,80,Charters,8065,University Heights Charter School,858,0,0,724,84.4,133,...,0,0,0,0,73.1,13.9,6.4,0,0,0


In [6]:
df1 = df1.rename(columns={'Unnamed: 0': 'County Code', 'Unnamed: 1': 'County Name', 'Unnamed: 2': 'District Code', 'Unnamed: 3': 'District Name', 'Unnamed: 4': 'Total Enrollment', 'Unnamed: 5': 'White', 'Unnamed: 6': '%White', 'Unnamed: 7': 'Black', 'Unnamed: 8': '%Black', 'Unnamed: 9': 'Hispanic', 'Unnamed: 10': '%Hispanic', 'Unnamed: 11': 'Asian', 'Unnamed: 12': '%Asian', 'Unnamed: 13': 'Native American', 'Unnamed: 14': '%Native American', 'Unnamed: 15': 'Hawaiian Native', 'Unnamed: 16': '%Hawaiian Native', 'Unnamed: 17': 'Two or More Races', 'Unnamed 18': '%Two or More Races'})

In [7]:
df1

,County Code,County Name,District Code,District Name,Total Enrollment,White,%White,Black,%Black,Hispanic,...,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41
1,County Code,County Name,District Code,District Name,Total Enrollment,White,%White,Black,%Black,Hispanic,...,Tenth Grade,Eleventh Grade,Twelfth Grade,Ungraded,%Free Lunch,%Reduced Lunch,%English Learners,%Migrant,%Military,%Homeless
2,01,Atlantic,0010,Absecon Public Schools District,887,447,50.4,133,15,208,...,0,0,0,25,40.8,5.9,8.7,0,0.8,0.2
3,01,Atlantic,0110,Atlantic City School District,6450,265,4.1,2147,33.3,2847,...,462,405,369,286,81.5,3.6,16.1,0,0,2.2
4,01,Atlantic,0120,Atlantic County Vocational School District,1713,516,30.1,296,17.3,721,...,399,355,362,160,48.6,9.6,0,0,2.5,0.6
5,01,Atlantic,0125,Atlantic County Special Services School District,306,95,31,82,26.8,108,...,3,5,13,244,39.5,5.9,0.3,0,0.3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,80,Charters,8010,Union County Teams Charter School-High School/...,400,1,0.3,242,60.5,153,...,33,34,39,0,61.3,10,4,0,0,4.8
668,80,Charters,8050,Unity Charter School,245,123,50.2,40,16.3,28,...,0,0,0,0,9.8,1.6,1.2,0,0,0
669,80,Charters,8060,University Academy Charter High School,439,18,4.1,240,54.7,142,...,118,105,100,0,53.3,9.8,0,0,0,0
670,80,Charters,8065,University Heights Charter School,858,0,0,724,84.4,133,...,0,0,0,0,73.1,13.9,6.4,0,0,0


In [8]:
df1=df1.rename(columns={'Unnamed: 18': '%Two or More Races'})

In [9]:
df1.drop(1,inplace=True)

In [10]:
df1

,County Code,County Name,District Code,District Name,Total Enrollment,White,%White,Black,%Black,Hispanic,...,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41
2,01,Atlantic,0010,Absecon Public Schools District,887,447,50.4,133,15,208,...,0,0,0,25,40.8,5.9,8.7,0,0.8,0.2
3,01,Atlantic,0110,Atlantic City School District,6450,265,4.1,2147,33.3,2847,...,462,405,369,286,81.5,3.6,16.1,0,0,2.2
4,01,Atlantic,0120,Atlantic County Vocational School District,1713,516,30.1,296,17.3,721,...,399,355,362,160,48.6,9.6,0,0,2.5,0.6
5,01,Atlantic,0125,Atlantic County Special Services School District,306,95,31,82,26.8,108,...,3,5,13,244,39.5,5.9,0.3,0,0.3,1
6,01,Atlantic,0570,Brigantine Public School District,444,314,70.7,18,4.1,62,...,0,0,0,12,33.8,8.3,3.4,0,2.3,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,80,Charters,8010,Union County Teams Charter School-High School/...,400,1,0.3,242,60.5,153,...,33,34,39,0,61.3,10,4,0,0,4.8
668,80,Charters,8050,Unity Charter School,245,123,50.2,40,16.3,28,...,0,0,0,0,9.8,1.6,1.2,0,0,0
669,80,Charters,8060,University Academy Charter High School,439,18,4.1,240,54.7,142,...,118,105,100,0,53.3,9.8,0,0,0,0
670,80,Charters,8065,University Heights Charter School,858,0,0,724,84.4,133,...,0,0,0,0,73.1,13.9,6.4,0,0,0


In [11]:
pd.set_option('display.max_columns', None)

In [12]:
df1

,County Code,County Name,District Code,District Name,Total Enrollment,White,%White,Black,%Black,Hispanic,%Hispanic,Asian,%Asian,Native American,%Native American,Hawaiian Native,%Hawaiian Native,Two or More Races,%Two or More Races,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41
2,01,Atlantic,0010,Absecon Public Schools District,887,447,50.4,133,15,208,23.4,57,6.4,0,0,0,0,42,4.7,0,48,0,83,94,76,107,88,84,94,94,94,0,0,0,0,25,40.8,5.9,8.7,0,0.8,0.2
3,01,Atlantic,0110,Atlantic City School District,6450,265,4.1,2147,33.3,2847,44.1,1046,16.2,1,0,42,0.7,102,1.6,0,340,0,379,482,429,472,440,457,473,487,473,496,462,405,369,286,81.5,3.6,16.1,0,0,2.2
4,01,Atlantic,0120,Atlantic County Vocational School District,1713,516,30.1,296,17.3,721,42.1,102,6,5,0.3,9,0.5,64,3.7,0,0,0,0,0,0,0,0,0,0,0,0,437,399,355,362,160,48.6,9.6,0,0,2.5,0.6
5,01,Atlantic,0125,Atlantic County Special Services School District,306,95,31,82,26.8,108,35.3,11,3.6,1,0.3,5,1.6,4,1.3,0,4,0,0,0,8,4,3,6,3,8,4,1,3,5,13,244,39.5,5.9,0.3,0,0.3,1
6,01,Atlantic,0570,Brigantine Public School District,444,314,70.7,18,4.1,62,14,48,10.8,0,0,0,0,2,0.5,0,56,0,34,38,46,39,41,34,42,58,44,0,0,0,0,12,33.8,8.3,3.4,0,2.3,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,80,Charters,8010,Union County Teams Charter School-High School/...,400,1,0.3,242,60.5,153,38.3,4,1,0,0,0,0,0,0,0,0,0,16,18,20,20,22,24,44,35,39,56,33,34,39,0,61.3,10,4,0,0,4.8
668,80,Charters,8050,Unity Charter School,245,123,50.2,40,16.3,28,11.4,37,15.1,0,0,0,0,17,6.9,0,0,0,28,25,29,25,25,28,30,23,32,0,0,0,0,0,9.8,1.6,1.2,0,0,0
669,80,Charters,8060,University Academy Charter High School,439,18,4.1,240,54.7,142,32.3,39,8.9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,116,118,105,100,0,53.3,9.8,0,0,0,0
670,80,Charters,8065,University Heights Charter School,858,0,0,724,84.4,133,15.5,1,0.1,0,0,0,0,0,0,0,119,0,73,82,89,115,106,66,76,63,69,0,0,0,0,0,73.1,13.9,6.4,0,0,0


In [13]:
df1.drop(['Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38', 'Unnamed: 39', 'Unnamed: 40', 'Unnamed: 41'],axis=1, inplace=True)

In [14]:
df1

,County Code,County Name,District Code,District Name,Total Enrollment,White,%White,Black,%Black,Hispanic,%Hispanic,Asian,%Asian,Native American,%Native American,Hawaiian Native,%Hawaiian Native,Two or More Races,%Two or More Races
2,01,Atlantic,0010,Absecon Public Schools District,887,447,50.4,133,15,208,23.4,57,6.4,0,0,0,0,42,4.7
3,01,Atlantic,0110,Atlantic City School District,6450,265,4.1,2147,33.3,2847,44.1,1046,16.2,1,0,42,0.7,102,1.6
4,01,Atlantic,0120,Atlantic County Vocational School District,1713,516,30.1,296,17.3,721,42.1,102,6,5,0.3,9,0.5,64,3.7
5,01,Atlantic,0125,Atlantic County Special Services School District,306,95,31,82,26.8,108,35.3,11,3.6,1,0.3,5,1.6,4,1.3
6,01,Atlantic,0570,Brigantine Public School District,444,314,70.7,18,4.1,62,14,48,10.8,0,0,0,0,2,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,80,Charters,8010,Union County Teams Charter School-High School/...,400,1,0.3,242,60.5,153,38.3,4,1,0,0,0,0,0,0
668,80,Charters,8050,Unity Charter School,245,123,50.2,40,16.3,28,11.4,37,15.1,0,0,0,0,17,6.9
669,80,Charters,8060,University Academy Charter High School,439,18,4.1,240,54.7,142,32.3,39,8.9,0,0,0,0,0,0
670,80,Charters,8065,University Heights Charter School,858,0,0,724,84.4,133,15.5,1,0.1,0,0,0,0,0,0


In [19]:
select_county = df1.loc[df1['County Name'] == 'Camden']
print (select_county)

    County Code County Name District Code  \
141          07      Camden          0150   
142          07      Camden          0190   
143          07      Camden          0260   
144          07      Camden          0330   
145          07      Camden          0340   
146          07      Camden          0390   
147          07      Camden          0580   
148          07      Camden          0680   
149          07      Camden          0700   
150          07      Camden          0800   
151          07      Camden          0880   
152          07      Camden          0940   
153          07      Camden          1255   
154          07      Camden          1720   
155          07      Camden          1770   
156          07      Camden          1780   
157          07      Camden          1799   
158          07      Camden          1801   
159          07      Camden          1802   
160          07      Camden          1880   
161          07      Camden          1890   
162       

In [20]:
df1=select_county

In [21]:
df1

,County Code,County Name,District Code,District Name,Total Enrollment,White,%White,Black,%Black,Hispanic,%Hispanic,Asian,%Asian,Native American,%Native American,Hawaiian Native,%Hawaiian Native,Two or More Races,%Two or More Races
141,07,Camden,0150,Audubon Public School District,1444,1235,85.5,47,3.3,109,7.5,16,1.1,5,0.3,2,0.1,30,2.1
142,07,Camden,0190,Barrington School District,563,432,76.7,29,5.2,48,8.5,24,4.3,3,0.5,3,0.5,24,4.3
143,07,Camden,0260,Bellmawr Public School District,1166,671,57.5,48,4.1,256,22,130,11.1,0,0,1,0.1,60,5.1
144,07,Camden,0330,Berlin Borough School District,807,659,81.7,57,7.1,26,3.2,29,3.6,2,0.2,0,0,34,4.2
145,07,Camden,0340,Berlin Township School District,604,356,58.9,73,12.1,114,18.9,52,8.6,0,0,0,0,9,1.5
146,07,Camden,0390,Black Horse Pike Regional School District,3478,1840,52.9,914,26.3,391,11.2,227,6.5,6,0.2,11,0.3,89,2.6
147,07,Camden,0580,Brooklawn Public School District,285,157,55.1,41,14.4,52,18.2,13,4.6,2,0.7,2,0.7,18,6.3
148,07,Camden,0680,Camden City School District,7343,68,0.9,3134,42.7,4006,54.6,69,0.9,2,0,14,0.2,50,0.7
149,07,Camden,0700,Camden County Technical School District,2245,664,29.6,538,24,884,39.4,77,3.4,3,0.1,1,0,78,3.5
150,07,Camden,0800,Cherry Hill School District,10469,5774,55.2,921,8.8,1414,13.5,1821,17.4,13,0.1,22,0.2,504,4.8


# 2. Dataset 2 Import

In [23]:
df2=pd.read_csv("https://github.com/XiaoLiang28/Datamanagement/raw/main/Teacher's%20Race%202021.csv")

In [24]:
df2

,Co Code,County,Dist Code,District,Position,White,%White,Black,%Black,Hispanic,%Hispanic,Asian,%Asian,American Indian,%American Indian,Hawaiian Native,%Hawaiian Native,Two or More Races,%Two or More Races,Total,Male,%Male,Female,%Female,%Non Binary
0,01,Atlantic,10.0,Absecon Public Schools District,Administrators,5.5,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.5,3.0,54.5,2.5,45.5,<.1
1,01,Atlantic,10.0,Absecon Public Schools District,Special Service,9.7,94.2,0.0,0.0,0.6,5.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.3,0.0,0.0,10.3,100.0,<.1
2,01,Atlantic,10.0,Absecon Public Schools District,Teacher,77.8,97.5,0.0,0.0,1.0,1.3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.3,79.8,15.0,18.8,64.8,81.2,<.1
3,01,Atlantic,10.0,Absecon Public Schools District,Supervisors/Coordinators,0.5,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.5,100.0,<.1
4,01,Atlantic,110.0,Atlantic City School District,Administrators,6.5,19.7,23.0,69.7,1.5,4.5,0.0,0.0,0.0,0.0,0.0,0.0,2.0,6.1,33.0,10.5,31.8,22.5,68.2,<.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2684,80,Charters,8140.0,The Village Charter School,Administrators,2.0,66.7,1.0,33.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,33.3,2.0,66.7,<.1
2685,80,Charters,8140.0,The Village Charter School,Special Service,0.0,0.0,3.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,3.0,100.0,<.1
2686,80,Charters,8140.0,The Village Charter School,Teacher,22.0,73.3,6.0,20.0,1.0,3.3,1.0,3.3,0.0,0.0,0.0,0.0,0.0,0.0,30.0,3.0,10.0,27.0,90.0,<.1
2687,80,Charters,8140.0,The Village Charter School,Supervisors/Coordinators,0.0,0.0,1.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,100.0,<.1


In [25]:
pd.set_option('display.max_columns', None)

In [26]:
df2

,Co Code,County,Dist Code,District,Position,White,%White,Black,%Black,Hispanic,%Hispanic,Asian,%Asian,American Indian,%American Indian,Hawaiian Native,%Hawaiian Native,Two or More Races,%Two or More Races,Total,Male,%Male,Female,%Female,%Non Binary
0,01,Atlantic,10.0,Absecon Public Schools District,Administrators,5.5,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.5,3.0,54.5,2.5,45.5,<.1
1,01,Atlantic,10.0,Absecon Public Schools District,Special Service,9.7,94.2,0.0,0.0,0.6,5.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.3,0.0,0.0,10.3,100.0,<.1
2,01,Atlantic,10.0,Absecon Public Schools District,Teacher,77.8,97.5,0.0,0.0,1.0,1.3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.3,79.8,15.0,18.8,64.8,81.2,<.1
3,01,Atlantic,10.0,Absecon Public Schools District,Supervisors/Coordinators,0.5,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.5,100.0,<.1
4,01,Atlantic,110.0,Atlantic City School District,Administrators,6.5,19.7,23.0,69.7,1.5,4.5,0.0,0.0,0.0,0.0,0.0,0.0,2.0,6.1,33.0,10.5,31.8,22.5,68.2,<.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2684,80,Charters,8140.0,The Village Charter School,Administrators,2.0,66.7,1.0,33.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,33.3,2.0,66.7,<.1
2685,80,Charters,8140.0,The Village Charter School,Special Service,0.0,0.0,3.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,3.0,100.0,<.1
2686,80,Charters,8140.0,The Village Charter School,Teacher,22.0,73.3,6.0,20.0,1.0,3.3,1.0,3.3,0.0,0.0,0.0,0.0,0.0,0.0,30.0,3.0,10.0,27.0,90.0,<.1
2687,80,Charters,8140.0,The Village Charter School,Supervisors/Coordinators,0.0,0.0,1.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,100.0,<.1


In [27]:
df2.drop(df2.tail(1).index,inplace=True)

In [28]:
df2

,Co Code,County,Dist Code,District,Position,White,%White,Black,%Black,Hispanic,%Hispanic,Asian,%Asian,American Indian,%American Indian,Hawaiian Native,%Hawaiian Native,Two or More Races,%Two or More Races,Total,Male,%Male,Female,%Female,%Non Binary
0,01,Atlantic,10.0,Absecon Public Schools District,Administrators,5.5,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.5,3.0,54.5,2.5,45.5,<.1
1,01,Atlantic,10.0,Absecon Public Schools District,Special Service,9.7,94.2,0.0,0.0,0.6,5.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.3,0.0,0.0,10.3,100.0,<.1
2,01,Atlantic,10.0,Absecon Public Schools District,Teacher,77.8,97.5,0.0,0.0,1.0,1.3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.3,79.8,15.0,18.8,64.8,81.2,<.1
3,01,Atlantic,10.0,Absecon Public Schools District,Supervisors/Coordinators,0.5,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.5,100.0,<.1
4,01,Atlantic,110.0,Atlantic City School District,Administrators,6.5,19.7,23.0,69.7,1.5,4.5,0.0,0.0,0.0,0.0,0.0,0.0,2.0,6.1,33.0,10.5,31.8,22.5,68.2,<.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2683,80,Charters,8065.0,University Heights Charter School,Supervisors/Coordinators,0.0,0.0,1.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,100.0,<.1
2684,80,Charters,8140.0,The Village Charter School,Administrators,2.0,66.7,1.0,33.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,33.3,2.0,66.7,<.1
2685,80,Charters,8140.0,The Village Charter School,Special Service,0.0,0.0,3.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,3.0,100.0,<.1
2686,80,Charters,8140.0,The Village Charter School,Teacher,22.0,73.3,6.0,20.0,1.0,3.3,1.0,3.3,0.0,0.0,0.0,0.0,0.0,0.0,30.0,3.0,10.0,27.0,90.0,<.1


In [29]:
df2 = df2.rename(columns={'Co Code': 'County Code'})

In [30]:
df2 = df2.rename(columns={'County': 'County Name', 'Dist Code': 'District Code', 'District': 'District Name'})

In [31]:
df2.drop(['Total', 'Male', '%Male', 'Female', '%Female', '%Non Binary'],axis=1, inplace=True)

In [32]:
df2 = df2.rename(columns={'American Indian': 'Native American', '%American Indian': '%Native American'})

In [33]:
df2

,County Code,County Name,District Code,District Name,Position,White,%White,Black,%Black,Hispanic,%Hispanic,Asian,%Asian,Native American,%Native American,Hawaiian Native,%Hawaiian Native,Two or More Races,%Two or More Races
0,01,Atlantic,10.0,Absecon Public Schools District,Administrators,5.5,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,01,Atlantic,10.0,Absecon Public Schools District,Special Service,9.7,94.2,0.0,0.0,0.6,5.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,01,Atlantic,10.0,Absecon Public Schools District,Teacher,77.8,97.5,0.0,0.0,1.0,1.3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.3
3,01,Atlantic,10.0,Absecon Public Schools District,Supervisors/Coordinators,0.5,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,01,Atlantic,110.0,Atlantic City School District,Administrators,6.5,19.7,23.0,69.7,1.5,4.5,0.0,0.0,0.0,0.0,0.0,0.0,2.0,6.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2683,80,Charters,8065.0,University Heights Charter School,Supervisors/Coordinators,0.0,0.0,1.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2684,80,Charters,8140.0,The Village Charter School,Administrators,2.0,66.7,1.0,33.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2685,80,Charters,8140.0,The Village Charter School,Special Service,0.0,0.0,3.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2686,80,Charters,8140.0,The Village Charter School,Teacher,22.0,73.3,6.0,20.0,1.0,3.3,1.0,3.3,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
selectcounty = df2.loc[df2['County Name'] == 'Camden']
print (selectcounty)

    County Code County Name  District Code                     District Name  \
556          07      Camden          150.0    Audubon Public School District   
557          07      Camden          150.0    Audubon Public School District   
558          07      Camden          150.0    Audubon Public School District   
559          07      Camden          150.0    Audubon Public School District   
560          07      Camden          190.0        Barrington School District   
..          ...         ...            ...                               ...   
711          07      Camden         5820.0  Winslow Township School District   
712          07      Camden         5900.0         Woodlynne School District   
713          07      Camden         5900.0         Woodlynne School District   
714          07      Camden         5900.0         Woodlynne School District   
715          07      Camden         5900.0         Woodlynne School District   

                     Position  White  %

In [35]:
df2=selectcounty
df2

,County Code,County Name,District Code,District Name,Position,White,%White,Black,%Black,Hispanic,%Hispanic,Asian,%Asian,Native American,%Native American,Hawaiian Native,%Hawaiian Native,Two or More Races,%Two or More Races
556,07,Camden,150.0,Audubon Public School District,Administrators,8.1,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
557,07,Camden,150.0,Audubon Public School District,Special Service,20.3,96.2,0.0,0.0,0.0,0.0,0.8,3.8,0.0,0.0,0.0,0.0,0.0,0.0
558,07,Camden,150.0,Audubon Public School District,Teacher,119.2,97.5,0.0,0.0,1.0,0.8,2.0,1.6,0.0,0.0,0.0,0.0,0.0,0.0
559,07,Camden,150.0,Audubon Public School District,Supervisors/Coordinators,1.8,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
560,07,Camden,190.0,Barrington School District,Administrators,4.9,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
711,07,Camden,5820.0,Winslow Township School District,Supervisors/Coordinators,2.0,66.7,1.0,33.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
712,07,Camden,5900.0,Woodlynne School District,Administrators,1.0,33.3,2.0,66.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
713,07,Camden,5900.0,Woodlynne School District,Special Service,3.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
714,07,Camden,5900.0,Woodlynne School District,Teacher,31.1,94.0,2.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# 3. Merge

In [ ]:
mergedOuter = pd.merge(df1, df2, how='outer', on=['County Name'],indicator=True)
mergedOuter

,County Code_x,County Name,District Code_x,District Name_x,Total Enrollment,White_x,%White_x,Black_x,%Black_x,Hispanic_x,%Hispanic_x,Asian_x,%Asian_x,Native American_x,%Native American_x,Hawaiian Native_x,%Hawaiian Native_x,Two or More Races_x,%Two or More Races_x,County Code_y,District Code_y,District Name_y,Position,White_y,%White_y,Black_y,%Black_y,Hispanic_y,%Hispanic_y,Asian_y,%Asian_y,Native American_y,%Native American_y,Hawaiian Native_y,%Hawaiian Native_y,Two or More Races_y,%Two or More Races_y,_merge
0,01,Atlantic,0010,Absecon Public Schools District,887,447,50.4,133,15,208,23.4,57,6.4,0,0,0,0,42,4.7,01,10.0,Absecon Public Schools District,Administrators,5.5,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,both
1,01,Atlantic,0010,Absecon Public Schools District,887,447,50.4,133,15,208,23.4,57,6.4,0,0,0,0,42,4.7,01,10.0,Absecon Public Schools District,Special Service,9.7,94.2,0.0,0.0,0.6,5.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,both
2,01,Atlantic,0010,Absecon Public Schools District,887,447,50.4,133,15,208,23.4,57,6.4,0,0,0,0,42,4.7,01,10.0,Absecon Public Schools District,Teacher,77.8,97.5,0.0,0.0,1.0,1.3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.3,both
3,01,Atlantic,0010,Absecon Public Schools District,887,447,50.4,133,15,208,23.4,57,6.4,0,0,0,0,42,4.7,01,10.0,Absecon Public Schools District,Supervisors/Coordinators,0.5,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,both
4,01,Atlantic,0010,Absecon Public Schools District,887,447,50.4,133,15,208,23.4,57,6.4,0,0,0,0,42,4.7,01,110.0,Atlantic City School District,Administrators,6.5,19.7,23.0,69.7,1.5,4.5,0.0,0.0,0.0,0.0,0.0,0.0,2.0,6.1,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113483,80,Charters,8140,The Village Charter School,363,4,1.1,345,95,14,3.9,0,0,0,0,0,0,0,0,80,8065.0,University Heights Charter School,Supervisors/Coordinators,0.0,0.0,1.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,both
113484,80,Charters,8140,The Village Charter School,363,4,1.1,345,95,14,3.9,0,0,0,0,0,0,0,0,80,8140.0,The Village Charter School,Administrators,2.0,66.7,1.0,33.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,both
113485,80,Charters,8140,The Village Charter School,363,4,1.1,345,95,14,3.9,0,0,0,0,0,0,0,0,80,8140.0,The Village Charter School,Special Service,0.0,0.0,3.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,both
113486,80,Charters,8140,The Village Charter School,363,4,1.1,345,95,14,3.9,0,0,0,0,0,0,0,0,80,8140.0,The Village Charter School,Teacher,22.0,73.3,6.0,20.0,1.0,3.3,1.0,3.3,0.0,0.0,0.0,0.0,0.0,0.0,both


In [ ]:
df1.dtypes

County Code           object
County Name           object
District Code         object
District Name         object
Total Enrollment      object
White                 object
%White                object
Black                 object
%Black                object
Hispanic              object
%Hispanic             object
Asian                 object
%Asian                object
Native American       object
%Native American      object
Hawaiian Native       object
%Hawaiian Native      object
Two or More Races     object
%Two or More Races    object
dtype: object

In [ ]:
mergedOuter['_merge'].value_counts()
mergedOuter['County Name'][mergedOuter['_merge']=='left_only']
mergedOuter['County Name'][mergedOuter['_merge']=='right_only']

both          113488
left_only          0
right_only         0
Name: _merge, dtype: int64

Series([], Name: County Name, dtype: object)

Series([], Name: County Name, dtype: object)

In [ ]:
mergedInner = pd.merge(df1, df2, how='inner', on=['County Name'])
mergedInner

,County Code_x,County Name,District Code_x,District Name_x,Total Enrollment,White_x,%White_x,Black_x,%Black_x,Hispanic_x,%Hispanic_x,Asian_x,%Asian_x,Native American_x,%Native American_x,Hawaiian Native_x,%Hawaiian Native_x,Two or More Races_x,%Two or More Races_x,County Code_y,District Code_y,District Name_y,Position,White_y,%White_y,Black_y,%Black_y,Hispanic_y,%Hispanic_y,Asian_y,%Asian_y,Native American_y,%Native American_y,Hawaiian Native_y,%Hawaiian Native_y,Two or More Races_y,%Two or More Races_y
0,01,Atlantic,0010,Absecon Public Schools District,887,447,50.4,133,15,208,23.4,57,6.4,0,0,0,0,42,4.7,01,10.0,Absecon Public Schools District,Administrators,5.5,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,01,Atlantic,0010,Absecon Public Schools District,887,447,50.4,133,15,208,23.4,57,6.4,0,0,0,0,42,4.7,01,10.0,Absecon Public Schools District,Special Service,9.7,94.2,0.0,0.0,0.6,5.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,01,Atlantic,0010,Absecon Public Schools District,887,447,50.4,133,15,208,23.4,57,6.4,0,0,0,0,42,4.7,01,10.0,Absecon Public Schools District,Teacher,77.8,97.5,0.0,0.0,1.0,1.3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.3
3,01,Atlantic,0010,Absecon Public Schools District,887,447,50.4,133,15,208,23.4,57,6.4,0,0,0,0,42,4.7,01,10.0,Absecon Public Schools District,Supervisors/Coordinators,0.5,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,01,Atlantic,0010,Absecon Public Schools District,887,447,50.4,133,15,208,23.4,57,6.4,0,0,0,0,42,4.7,01,110.0,Atlantic City School District,Administrators,6.5,19.7,23.0,69.7,1.5,4.5,0.0,0.0,0.0,0.0,0.0,0.0,2.0,6.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113483,80,Charters,8140,The Village Charter School,363,4,1.1,345,95,14,3.9,0,0,0,0,0,0,0,0,80,8065.0,University Heights Charter School,Supervisors/Coordinators,0.0,0.0,1.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
113484,80,Charters,8140,The Village Charter School,363,4,1.1,345,95,14,3.9,0,0,0,0,0,0,0,0,80,8140.0,The Village Charter School,Administrators,2.0,66.7,1.0,33.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
113485,80,Charters,8140,The Village Charter School,363,4,1.1,345,95,14,3.9,0,0,0,0,0,0,0,0,80,8140.0,The Village Charter School,Special Service,0.0,0.0,3.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
113486,80,Charters,8140,The Village Charter School,363,4,1.1,345,95,14,3.9,0,0,0,0,0,0,0,0,80,8140.0,The Village Charter School,Teacher,22.0,73.3,6.0,20.0,1.0,3.3,1.0,3.3,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
mergedLeft = pd.merge(df1, df2, how='left', on=['County Name'])
mergedLeft

,County Code_x,County Name,District Code_x,District Name_x,Total Enrollment,White_x,%White_x,Black_x,%Black_x,Hispanic_x,%Hispanic_x,Asian_x,%Asian_x,Native American_x,%Native American_x,Hawaiian Native_x,%Hawaiian Native_x,Two or More Races_x,%Two or More Races_x,County Code_y,District Code_y,District Name_y,Position,White_y,%White_y,Black_y,%Black_y,Hispanic_y,%Hispanic_y,Asian_y,%Asian_y,Native American_y,%Native American_y,Hawaiian Native_y,%Hawaiian Native_y,Two or More Races_y,%Two or More Races_y
0,01,Atlantic,0010,Absecon Public Schools District,887,447,50.4,133,15,208,23.4,57,6.4,0,0,0,0,42,4.7,01,10.0,Absecon Public Schools District,Administrators,5.5,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,01,Atlantic,0010,Absecon Public Schools District,887,447,50.4,133,15,208,23.4,57,6.4,0,0,0,0,42,4.7,01,10.0,Absecon Public Schools District,Special Service,9.7,94.2,0.0,0.0,0.6,5.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,01,Atlantic,0010,Absecon Public Schools District,887,447,50.4,133,15,208,23.4,57,6.4,0,0,0,0,42,4.7,01,10.0,Absecon Public Schools District,Teacher,77.8,97.5,0.0,0.0,1.0,1.3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.3
3,01,Atlantic,0010,Absecon Public Schools District,887,447,50.4,133,15,208,23.4,57,6.4,0,0,0,0,42,4.7,01,10.0,Absecon Public Schools District,Supervisors/Coordinators,0.5,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,01,Atlantic,0010,Absecon Public Schools District,887,447,50.4,133,15,208,23.4,57,6.4,0,0,0,0,42,4.7,01,110.0,Atlantic City School District,Administrators,6.5,19.7,23.0,69.7,1.5,4.5,0.0,0.0,0.0,0.0,0.0,0.0,2.0,6.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113483,80,Charters,8140,The Village Charter School,363,4,1.1,345,95,14,3.9,0,0,0,0,0,0,0,0,80,8065.0,University Heights Charter School,Supervisors/Coordinators,0.0,0.0,1.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
113484,80,Charters,8140,The Village Charter School,363,4,1.1,345,95,14,3.9,0,0,0,0,0,0,0,0,80,8140.0,The Village Charter School,Administrators,2.0,66.7,1.0,33.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
113485,80,Charters,8140,The Village Charter School,363,4,1.1,345,95,14,3.9,0,0,0,0,0,0,0,0,80,8140.0,The Village Charter School,Special Service,0.0,0.0,3.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
113486,80,Charters,8140,The Village Charter School,363,4,1.1,345,95,14,3.9,0,0,0,0,0,0,0,0,80,8140.0,The Village Charter School,Teacher,22.0,73.3,6.0,20.0,1.0,3.3,1.0,3.3,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
mergedRight = pd.merge(df1, df2, how='right', on=['County Name'])
mergedRight

,County Code_x,County Name,District Code_x,District Name_x,Total Enrollment,White_x,%White_x,Black_x,%Black_x,Hispanic_x,%Hispanic_x,Asian_x,%Asian_x,Native American_x,%Native American_x,Hawaiian Native_x,%Hawaiian Native_x,Two or More Races_x,%Two or More Races_x,County Code_y,District Code_y,District Name_y,Position,White_y,%White_y,Black_y,%Black_y,Hispanic_y,%Hispanic_y,Asian_y,%Asian_y,Native American_y,%Native American_y,Hawaiian Native_y,%Hawaiian Native_y,Two or More Races_y,%Two or More Races_y
0,01,Atlantic,0010,Absecon Public Schools District,887,447,50.4,133,15,208,23.4,57,6.4,0,0,0,0,42,4.7,01,10.0,Absecon Public Schools District,Administrators,5.5,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,01,Atlantic,0110,Atlantic City School District,6450,265,4.1,2147,33.3,2847,44.1,1046,16.2,1,0,42,0.7,102,1.6,01,10.0,Absecon Public Schools District,Administrators,5.5,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,01,Atlantic,0120,Atlantic County Vocational School District,1713,516,30.1,296,17.3,721,42.1,102,6,5,0.3,9,0.5,64,3.7,01,10.0,Absecon Public Schools District,Administrators,5.5,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,01,Atlantic,0125,Atlantic County Special Services School District,306,95,31,82,26.8,108,35.3,11,3.6,1,0.3,5,1.6,4,1.3,01,10.0,Absecon Public Schools District,Administrators,5.5,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,01,Atlantic,0570,Brigantine Public School District,444,314,70.7,18,4.1,62,14,48,10.8,0,0,0,0,2,0.5,01,10.0,Absecon Public Schools District,Administrators,5.5,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113483,80,Charters,8010,Union County Teams Charter School-High School/...,400,1,0.3,242,60.5,153,38.3,4,1,0,0,0,0,0,0,80,8140.0,The Village Charter School,Supervisors/Coordinators,0.0,0.0,1.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
113484,80,Charters,8050,Unity Charter School,245,123,50.2,40,16.3,28,11.4,37,15.1,0,0,0,0,17,6.9,80,8140.0,The Village Charter School,Supervisors/Coordinators,0.0,0.0,1.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
113485,80,Charters,8060,University Academy Charter High School,439,18,4.1,240,54.7,142,32.3,39,8.9,0,0,0,0,0,0,80,8140.0,The Village Charter School,Supervisors/Coordinators,0.0,0.0,1.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
113486,80,Charters,8065,University Heights Charter School,858,0,0,724,84.4,133,15.5,1,0.1,0,0,0,0,0,0,80,8140.0,The Village Charter School,Supervisors/Coordinators,0.0,0.0,1.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
